In [ ]:
#conda install hoomd
#pip install numpy matplotlib pymbar

In [1]:
import hoomd
import hoomd.md
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ---------------------------
# 1) Physical constants (placeholder)
# ---------------------------
kB = 1.380649e-23       # Boltzmann constant (J/K)
NA = 6.02214076e23      # Avogadro's number (1/mol)
# Additional unit conversions if needed...

In [3]:
# ---------------------------
# 2) Toy LJ parameters and reference scales
# ---------------------------
# Reference temperature and sigma
Tref = 300.0       # [K]
sigma_ref = 3.50   # [Å] for dimensionless sigma

# Example LJ "benzene" (very rough) and "methane" parameters in K
eps_benzene_kB  = 80.0
eps_methane_kB  = 148.0
eps_cross_kB    = np.sqrt(eps_methane_kB * eps_benzene_kB)

sigma_benzene_A = 3.50
sigma_methane_A = 3.73
sigma_cross_A   = 0.5 * (sigma_benzene_A + sigma_methane_A)

# Convert these to dimensionless
eps_benzene_star  = eps_benzene_kB  / Tref
eps_methane_star  = eps_methane_kB  / Tref
eps_cross_star    = eps_cross_kB    / Tref

sigma_benzene_star = sigma_benzene_A / sigma_ref
sigma_methane_star = sigma_methane_A / sigma_ref
sigma_cross_star   = sigma_cross_A   / sigma_ref

In [4]:
# ---------------------------
# 3) Initialize HOOMD
# ---------------------------
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=hoomd.device.CPU())
sim.seed = 42

In [5]:
# ---------------------------
# 4) Define the simulation box & initial snapshot
# ---------------------------
box_length = 100.0  # dimensionless box length

snap = hoomd.Snapshot()
snap.configuration.box = [box_length, box_length, box_length, 0, 0, 0]

N_benzene = 500
N_methane = 500
N_total = N_benzene + N_methane

snap.particles.N = N_total
snap.particles.types = ['benzene', 'methane']

# Random positions with smaller std dev to reduce overlap
benzene_positions = np.random.normal(loc=0.0, scale=box_length*0.02, size=(N_benzene, 3))
methane_positions = np.random.normal(loc=0.0, scale=box_length*0.02, size=(N_methane, 3))
positions = np.vstack([benzene_positions, methane_positions])

snap.particles.position[:] = positions
snap.particles.typeid[:N_benzene] = 0
snap.particles.typeid[N_benzene:] = 1

# Wrap to ensure all coordinates lie within [-L/2, +L/2]
snap.wrap()

# Create the simulation state
sim.create_state_from_snapshot(snap)


In [ ]:
# ---------------------------
# 5) Define forces and a short minimization
# ---------------------------
nl = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=nl)

cutoff = 2.5
lj.r_cut[('benzene', 'benzene')] = cutoff * sigma_benzene_star
lj.r_cut[('methane', 'methane')] = cutoff * sigma_methane_star
lj.r_cut[('benzene', 'methane')] = cutoff * sigma_cross_star

lj.params[('benzene', 'benzene')] = dict(epsilon=eps_benzene_star, sigma=sigma_benzene_star)
lj.params[('methane', 'methane')] = dict(epsilon=eps_methane_star, sigma=sigma_methane_star)
lj.params[('benzene', 'methane')] = dict(epsilon=eps_cross_star,   sigma=sigma_cross_star)

# We'll do a FIRE minimization first to remove any huge overlaps
fire_integrator = hoomd.md.Integrator(dt=0.0005)
fire_integrator.forces.append(lj)  # "lj" is your pair force

fire = hoomd.md.minimize.FIRE(
    dt=0.0005,
    force_tol=1e-5,         # force convergence criterion
    angmom_tol=1e-5,        # angular momentum convergence criterion
    energy_tol=1e-5,        # energy change convergence criterion
    integrate_rotational_dof=False,
    forces=fire_integrator.forces,
    min_steps_conv=100
)

sim.operations.integrator = fire_integrator

# Perform the minimization
print("Minimizing...")
while not fire.converged:
    sim.run(100)

Minimizing...


In [ ]:
# ---------------------------
# 6) Set up a "ConstantPressure" integrator (NPT-like)
# ---------------------------
# For the toy VLE approach, we'll still run some displacements at each T, but 
# let's use an NPT method to allow volume changes (if you prefer NVT, see below).

npt_thermostat = hoomd.md.methods.thermostats.MTTK(kT=1.0, tau=1.0)  # placeholder kT, will set later
npt = hoomd.md.methods.ConstantPressure(
    filter=hoomd.filter.All(),
    thermostat=npt_thermostat,
    S=0.1,   # dimensionless pressure placeholder, will set later
    tauS=1.0,
    couple='xyz'
)

npt_integrator = hoomd.md.Integrator(dt=0.0005)
npt_integrator.forces.append(lj)
npt_integrator.methods.append(npt)
sim.operations.integrator = npt_integrator

In [ ]:
# ---------------------------
# 7) Toy "Gibbs Ensemble MC" style displacement
# ---------------------------
def get_total_potential_energy(sim):
    """Compute the total potential energy by forcing a 0-step run."""
    sim.run(0)  # triggers energy computation
    total_energy = 0.0
    for f in sim.operations.integrator.forces:
        total_energy += f.energy
    return total_energy

def wrap_position(pos, box):
    """Wrap a 3D position into a periodic orthorhombic box."""
    x, y, z = pos
    Lx, Ly, Lz = box.Lx, box.Ly, box.Lz
    # Shift from (-L/2, L/2) to [0, L), apply modulo, shift back
    x = ((x + 0.5 * Lx) % Lx) - 0.5 * Lx
    y = ((y + 0.5 * Ly) % Ly) - 0.5 * Ly
    z = ((z + 0.5 * Lz) % Lz) - 0.5 * Lz
    return (x, y, z)

def gibbs_ensemble_mc(sim, n_steps, temperature, max_displacement=0.1, kB_hoomd=1.0):
    """
    Toy MC displacement moves in an otherwise MD context.
    `kB_hoomd` is HOOMD's dimensionless Boltzmann constant (often 1.0 if fully reduced).
    """
    for _ in range(int(n_steps)):
        snap = sim.state.get_snapshot()
        if snap.particles is None:
            continue

        N = snap.particles.N
        particle_idx = np.random.randint(0, N)

        old_position = snap.particles.position[particle_idx]
        displacement = np.random.uniform(-max_displacement, max_displacement, 3)
        new_position = old_position + displacement

        # Keep it in the box
        box = sim.state.box
        new_position = wrap_position(new_position, box)

        old_energy = get_total_potential_energy(sim)

        # Attempt the trial move
        snap.particles.position[particle_idx] = new_position
        sim.state.set_snapshot(snap)

        new_energy = get_total_potential_energy(sim)
        delta_energy = new_energy - old_energy

        # Metropolis acceptance: dimensionless energy vs dimensionless T
        # Usually k_B = 1 in reduced units, so "delta_energy / (kB_hoomd * temperature)"
        if delta_energy < 0:
            accept = True
        else:
            if np.random.rand() < np.exp(-delta_energy / (kB_hoomd * temperature)):
                accept = True
            else:
                accept = False

        if not accept:
            # revert
            snap = sim.state.get_snapshot()
            if snap.particles is not None:
                snap.particles.position[particle_idx] = old_position
                sim.state.set_snapshot(snap)

In [ ]:
# ---------------------------
# 8) Compute "VLE-like" properties
# ---------------------------
def calculate_vle_properties(sim, temperatures, pressures, npt_method, steps_md=1e4, steps_mc=1000):
    """Runs MD + toy MC moves at each (T, P), then measures densities (or z-coord means)."""
    densities_liquid = []
    densities_vapor = []

    for T, P in zip(temperatures, pressures):
        # Set the dimensionless T, P for the barostat/thermostat
        T_star = T / Tref
        P_star = P * 0.01  # Toy conversion: e.g. if we want P from 1..10 atm => 0.01..0.1 dimensionless
                           # In reality, do correct conversions: p* = p sigma^3 / (epsilon).
        npt_method.thermostat.kT = T_star
        npt_method.S = P_star

        # Re-equilibrate with MD
        sim.run(int(steps_md))

        # Toy MC displacements
        gibbs_ensemble_mc(sim, n_steps=steps_mc, temperature=T_star, max_displacement=0.1, kB_hoomd=1.0)

        # Measure "liquid" vs "vapor" by z-coordinate for first 500 vs last 500
        snap = sim.state.get_snapshot()
        if snap.particles is None:
            densities_liquid.append(0.0)
            densities_vapor.append(0.0)
        else:
            pos = snap.particles.position
            z_coords_benzene = pos[:500, 2]
            z_coords_methane = pos[500:, 2]
            density_liquid = np.mean(z_coords_benzene)   # Not a real density, just a "toy" measure
            density_vapor  = np.mean(z_coords_methane)
            densities_liquid.append(density_liquid)
            densities_vapor.append(density_vapor)

    return densities_liquid, densities_vapor

In [ ]:
# ---------------------------
# 9) Plot results
# ---------------------------
def plot_vle_diagram(temperatures, pressures, densities_liquid, densities_vapor):
    plt.figure(figsize=(8, 6))
    plt.plot(temperatures, pressures, 'o-', label='Pressures vs T')
    plt.xlabel('Temperature [K]')
    plt.ylabel('Pressure [atm] (toy scale)')
    plt.title('Toy Vapor-Liquid Equilibrium Phase Diagram')
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
# ---------------------------
# MAIN SCRIPT: short run + generate "VLE-like" data
# ---------------------------

print("\nStarting short NPT run to stabilize...\n")
sim.run(2_0000)  # 20k steps

# Suppose we want T from 300..500 K, P from 1..10 atm
temperatures = np.linspace(300, 500, 5)
pressures = np.linspace(1, 10, 5)  # 1..10 atm (toy labeling)

densities_liquid, densities_vapor = calculate_vle_properties(
    sim, temperatures, pressures, npt, steps_md=1e4, steps_mc=1000
)

plot_vle_diagram(temperatures, pressures, densities_liquid, densities_vapor)